知识点

df.filter(regex='880*',axis=0) # 对index进行筛选，正则表达

In [1]:
%matplotlib inline

In [2]:
import QUANTAXIS as QA
import pandas as pd
import tushare as ts
import numpy as np
# import TA-Lib as ta
import scipy.signal as signal
import matplotlib.pyplot as plt

In [3]:
import tushare as ts
token = "488917d460bca13177c0b510492fc5fc04d0629bc76cc32a4f73fcf0"
ts.set_token(token)
pro = ts.pro_api()

In [4]:
import pandasql as sqldf

In [5]:
#超大板块
# 电气设备    装备制造 查看相关性高的板块
# 酿酒      46434.23
# 化工      44198.02
# 医药      40178.99
# 汽车类     34589.40
# 元器件     33136.58
supper_big_industry_code = ['880471','880446','880380','880335','880400','880390','880492']
# ['880380' '880335' '880400' '880390'] '880471','880446','880492'

# 数据准备

## 获得龙头

In [190]:
# 对板块进行分类
blk_category = pd.read_csv('blk_category.csv').drop('Unnamed: 0',axis=1).set_index('细分行业')

In [6]:
detail_industry = pd.read_csv('申万细分行业.csv')

In [7]:
detail_industry.loc[:,('blk_code')] = detail_industry['代码'].astype(str)

In [8]:
all_stock_basic_raw = pd.read_csv(r"20220916.csv",encoding='utf-8-sig')[:-1]

In [9]:
# 数据清洗
all_stock_basic_raw.loc[:,'ab_mv_raw'] = all_stock_basic_raw['AB股总市值'].map(lambda x: x.strip().replace('亿',''))
all_stock_basic_raw.loc[:,'circulate_mv_raw'] = all_stock_basic_raw['流通市值'].map(lambda x: x.strip().replace('亿',''))
all_stock_basic = all_stock_basic_raw[all_stock_basic_raw['ab_mv_raw'] != '--']
# 数据处理
all_stock_basic.loc[:,'parent_industry'] = all_stock_basic['所属行业'].map(lambda x: x.strip().split('-')[0])
all_stock_basic.loc[:,'child_industry'] = all_stock_basic['所属行业'].map(lambda x: x.strip().split('-')[1])
all_stock_basic.loc[:,'ab_mv'] = all_stock_basic['ab_mv_raw'].map(lambda x: pd.to_numeric(x))
all_stock_basic.loc[:,'circulate_mv'] = all_stock_basic['circulate_mv_raw'].map(lambda x: pd.to_numeric(x))

d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [153]:
# 数据分析
# blk_mv = pd.DataFrame(all_stock_basic.groupby(['细分行业'])['ab_mv'].agg(['sum','count'])).rename(columns={'sum':'blk_mv'}) 
blk_mv_cnt = pd.DataFrame(all_stock_basic.groupby(['细分行业'])['circulate_mv'].agg(['sum','count'])).rename(columns={'sum':'blk_mv'})
detail_industry_name_code_map = detail_industry.filter(['名称','blk_code']).set_index('名称')
blk_info = pd.concat([blk_mv_cnt,detail_industry_name_code_map],axis=1)
all_stock_basic_1 = all_stock_basic.merge(blk_info,left_on = '细分行业',right_index=True)
all_stock_basic_1.loc[:,'mv_share'] = all_stock_basic_1['ab_mv']/all_stock_basic_1['blk_mv']*100
all_stock_basic_1.loc[:,'circulate_share'] = all_stock_basic_1['circulate_mv']/all_stock_basic_1['blk_mv']*100

In [193]:
detail_blk_basics = blk_mv_cnt.join(detail_industry.drop('代码',axis=1).set_index('名称')).join(blk_category)

In [195]:
detail_industry_code_name_map

,名称
blk_code,
880313,石油贸易
880363,林业
880431,船舶
880490,通信设备
880444,农用机械
...,...
880302,煤炭开采
880373,乳制品
880351,矿物制品


In [194]:
detail_blk_basics

,blk_mv,count,blk_code,category
细分行业,,,,
IT设备,3204.47,44,880489,中小型板块
专用机械,12506.07,218,880445,大板块
中成药,8239.00,74,880403,中型板块
乳制品,2737.83,16,880373,中小型板块
互联网,5077.39,78,880494,中型板块
...,...,...,...,...
银行,59285.89,42,880471,超大板块
陶瓷,468.70,7,880345,小型板块
食品,10395.41,89,880375,大板块


In [11]:
all_stock_basic_1.loc[:,'stock_code'] = all_stock_basic_1['代码']

In [12]:
all_stock_basic_data = all_stock_basic_1.set_index('stock_code')

In [13]:
all_stock_basic_data.loc[:,'circulate_share_rk'] = all_stock_basic_data.groupby(['细分行业'])['circulate_share']\
                                            .rank(ascending=0,method='dense')

In [14]:
all_stock_basic_data.query('细分行业=="银行"').filter(['circulate_mv','circulate_share_rk']).sort_values(by='circulate_share_rk',ascending = 1)

,circulate_mv,circulate_share_rk
stock_code,,
601398,11755.09,1.0
601288,8529.61,2.0
600036,7209.82,3.0
601988,6428.35,4.0
601166,3577.32,5.0
000001,2437.34,6.0
600000,2095.75,7.0
002142,2061.24,8.0
601328,1805.54,9.0


In [15]:
# all_stock_basic_data.groupby(['细分行业'])[['mv_share']].rank(ascending=0,method='dense')

In [16]:
# all_stock_basic_data.groupby(['细分行业'])[['mv_share']].rank(ascending=1,method='dense')

In [17]:
# 龙头
top1_stocks_all = all_stock_basic_data.groupby(['细分行业'])\
    .apply(lambda x:x.sort_values(by='mv_share',ascending=False).head(1))\
    .filter(['代码','名称','ab_mv','blk_mv','mv_share','count']).sort_values(by='count',ascending=False)

In [18]:
import numpy as np

### 创建板块成分股票

In [19]:
# all_blk_component = all_stock_basic_1.filter(['代码','细分行业','blk_code']).astype(str)\
#         .groupby(['细分行业','blk_code'])['代码'].apply(lambda x:  ",".join(x).split(','))
all_blk_component = all_stock_basic_1.filter(['代码','细分行业','blk_code'])\
        .groupby(['细分行业','blk_code']).agg(list)['代码']
all_blk_component

细分行业  blk_code
IT设备  880489      [000021, 000066, 000977, 001229, 001339, 00215...
专用机械  880445      [000519, 000551, 000584, 000856, 000925, 00098...
中成药   880403      [000423, 000538, 000590, 000623, 000650, 00079...
乳制品   880373      [001318, 002329, 002570, 002719, 002732, 00291...
互联网   880494      [000606, 000676, 000681, 000971, 002095, 00211...
                                        ...                        
银行    880471      [000001, 001227, 002142, 002807, 002839, 00293...
陶瓷    880345      [002162, 003012, 300234, 300285, 300409, 60326...
食品    880375      [000505, 000576, 000639, 000716, 000833, 00089...
饲料    880364      [000702, 000876, 001313, 002100, 002124, 00231...
黄金    880328      [000506, 000975, 002155, 002237, 600311, 60048...
Name: 代码, Length: 110, dtype: object

In [20]:
all_blk_component

细分行业  blk_code
IT设备  880489      [000021, 000066, 000977, 001229, 001339, 00215...
专用机械  880445      [000519, 000551, 000584, 000856, 000925, 00098...
中成药   880403      [000423, 000538, 000590, 000623, 000650, 00079...
乳制品   880373      [001318, 002329, 002570, 002719, 002732, 00291...
互联网   880494      [000606, 000676, 000681, 000971, 002095, 00211...
                                        ...                        
银行    880471      [000001, 001227, 002142, 002807, 002839, 00293...
陶瓷    880345      [002162, 003012, 300234, 300285, 300409, 60326...
食品    880375      [000505, 000576, 000639, 000716, 000833, 00089...
饲料    880364      [000702, 000876, 001313, 002100, 002124, 00231...
黄金    880328      [000506, 000975, 002155, 002237, 600311, 60048...
Name: 代码, Length: 110, dtype: object

In [21]:
all_blk_component.loc['专用机械'].values[0]

['000519',
 '000551',
 '000584',
 '000856',
 '000925',
 '000988',
 '001288',
 '001336',
 '002006',
 '002204',
 '002255',
 '002366',
 '002529',
 '002534',
 '002595',
 '002613',
 '002630',
 '002639',
 '002645',
 '002651',
 '002686',
 '002690',
 '002691',
 '002730',
 '002757',
 '002786',
 '002816',
 '002833',
 '002837',
 '002884',
 '002957',
 '002960',
 '002975',
 '002979',
 '002997',
 '300024',
 '300084',
 '300092',
 '300097',
 '300145',
 '300151',
 '300193',
 '300201',
 '300210',
 '300220',
 '300227',
 '300263',
 '300276',
 '300278',
 '300281',
 '300293',
 '300316',
 '300382',
 '300400',
 '300402',
 '300411',
 '300415',
 '300450',
 '300457',
 '300461',
 '300471',
 '300472',
 '300480',
 '300486',
 '300499',
 '300509',
 '300512',
 '300521',
 '300526',
 '300527',
 '300540',
 '300545',
 '300549',
 '300604',
 '300607',
 '300619',
 '300707',
 '300724',
 '300751',
 '300757',
 '300776',
 '300802',
 '300812',
 '300818',
 '300823',
 '300833',
 '300836',
 '300837',
 '300853',
 '300865',
 '300875',

### 非小型板块龙头

In [22]:
# 筛选非小板块
top1_stocks_non_small = top1_stocks_all.query('blk_mv>3000 and count>4').droplevel(level=1)
top1_stocks_non_small

,代码,名称,ab_mv,blk_mv,mv_share,count
细分行业,,,,,,
软件服务,688111,金山办公,1019.25,16617.04,6.133764,274
电气设备,300750,宁德时代,10005.93,44930.77,22.269661,263
元器件,002475,立讯精密,2421.81,25774.33,9.396209,252
化工原料,600309,万华化学,2837.70,21309.43,13.316640,227
专用机械,300316,晶盛机电,964.52,12506.07,7.712415,218
汽车配件,601689,拓普集团,833.26,13803.69,6.036502,200
医疗保健,300760,迈瑞医疗,3616.71,16474.48,21.953409,157
通信设备,601138,工业富联,1811.33,11711.94,15.465670,139
化学制药,600276,恒瑞医药,2221.81,14614.56,15.202716,136


### 小型板块龙头

In [23]:
top1_stocks_small_blk  = top1_stocks_all.query('blk_mv < 3000').droplevel(level=1)
top1_stocks_small_blk

,代码,名称,ab_mv,blk_mv,mv_share,count
细分行业,,,,,,
服饰,300979,华利集团,615.71,2708.84,22.729655,65
家居用品,603833,欧派家居,796.77,2684.91,29.675855,59
广告包装,002027,分众传媒,776.99,2810.51,27.645872,47
文教休闲,603899,晨光股份,415.27,1773.97,23.409077,45
纺织,603055,台华新材,100.08,1383.26,7.235082,44
运输设备,601766,中国中车,1184.76,2945.98,40.216159,41
供气供热,600803,新奥股份,566.75,2161.23,26.223493,41
百货,600655,豫园股份,280.16,2191.89,12.781663,38
其他建材,002271,东方雨虹,755.93,2300.11,32.864950,33


## 获取所有细分行业数据

In [24]:
# 转换code 为str
detail_industry_code = detail_industry['blk_code'].to_list()

In [25]:
#获取板块数据
detail_industry_data = QA.QA_fetch_index_day_adv(detail_industry_code,'2022-01-01','2022-09-16')

## 获取主要指数数据

In [26]:
index_code_name_map = pd.DataFrame(['上证指数','深证综指','上证50','中证500','创业板指','科创50','深证成指','沪深300'],\
                                   index = ['000001','399106','000016','000905','399006','000688','399001','399300'],columns=['名称'])

In [27]:
# 获取主要指数数据
main_index_data = QA.QA_fetch_index_day_adv(['000001','399106','000016','000905','399006','000688','399001','399300'],'2022-01-01','2022-09-16')

# 细分行业分析

In [28]:
detail_industry

,代码,名称,blk_code
0,880313,石油贸易,880313
1,880363,林业,880363
2,880431,船舶,880431
3,880490,通信设备,880490
4,880444,农用机械,880444
...,...,...,...
105,880302,煤炭开采,880302
106,880373,乳制品,880373
107,880351,矿物制品,880351
108,880382,啤酒,880382


In [29]:
detail_industry_code_name_map = detail_industry.drop(['代码'],axis=1).set_index('blk_code')

In [30]:
# detail_industry_code_name_map

In [31]:
#获取细分行业的板块名
index_detail_industry_code_name_map = pd.concat([index_code_name_map,detail_industry_code_name_map])

In [32]:
# super_big_industry_data = QA.QA_fetch_index_day_adv(supper_big_industry_code,'2022-01-01','2022-09-02')

In [33]:
# supper_and_detail_industry = pd.concat([super_big_industry_data.pivot('close'),detail_industry_data_new],axis=1)

In [34]:
# supper_and_detail_industry_corr = supper_and_detail_industry.pct_change()
# detail_industry_corr = detail_industry_data.pivot('close').pct_change().corr()
# 获得主要指数和板块的相关性
detail_industry_corr = pd.concat([detail_industry_data.pivot('close').pct_change(),main_index_data.pivot('close').pct_change()],axis=1)\
                            .corr()

In [36]:
blk_category.set_index

<bound method DataFrame.set_index of      category
细分行业         
银行       超大板块
电气设备     超大板块
白酒       超大板块
元器件      超大板块
化工原料     超大板块
...       ...
电器连锁     小型板块
公共交通     小型板块
渔业       小型板块
林业       小型板块
公路       小型板块

[110 rows x 1 columns]>

## 主要指数相关性分析

### 上证指数

In [37]:
# blk_mv

In [38]:
# 上证
index_code = '000001'
one_blk_corr = detail_industry_corr[index_code].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
main_index_raw = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
main_index = main_index_raw.merge(blk_basics,left_on='名称',right_index=True)
main_index.loc[:,'related_mv'] =  main_index.apply(lambda x: x[index_code]*x['blk_mv'],axis=1)
main_index.insert(1,'index',index_code)
main_index

,000001,index,名称,category,blk_mv,count,related_mv
code,,,,,,,
880432,0.884474,000001,运输设备,中小型板块,2945.98,41,2605.642192
880456,0.835621,000001,环境保护,中型板块,5054.20,125,4223.396604
880336,0.820566,000001,化工原料,超大板块,21309.43,227,17485.798600
880490,0.817318,000001,通信设备,大板块,11711.94,139,9572.373702
880421,0.817069,000001,广告包装,中小型板块,2810.51,47,2296.380950
880411,0.816062,000001,其他商业,小型板块,733.79,12,598.818324
880447,0.815734,000001,工程机械,中型板块,4301.18,27,3508.616727
880414,0.813418,000001,商贸代理,小型板块,1585.77,27,1289.894387
880474,0.811742,000001,多元金融,中小型板块,3414.14,30,2771.399551


In [39]:
上证影响力 = main_index.groupby(['index'])['related_mv'].sum()
上证影响力

index
000001    222242.552518
Name: related_mv, dtype: float64

### 深证综指

In [40]:
# 深证综指
index_code = '399106'
one_blk_corr = detail_industry_corr[index_code].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
main_index_raw = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
main_index = main_index_raw.merge(blk_basics,left_on='名称',right_index=True)
main_index.loc[:,'related_mv'] =  main_index.apply(lambda x: x[index_code]*x['blk_mv'],axis=1)
main_index.insert(1,'index',index_code)
main_index

,399106,index,名称,category,blk_mv,count,related_mv
code,,,,,,,
880490,0.907895,399106,通信设备,大板块,11711.94,139,10633.212028
880492,0.901120,399106,元器件,超大板块,25774.33,252,23225.765861
880445,0.886451,399106,专用机械,大板块,12506.07,218,11086.019451
880336,0.885723,399106,化工原料,超大板块,21309.43,227,18874.261981
880432,0.879878,399106,运输设备,中小型板块,2945.98,41,2592.102088
880456,0.868937,399106,环境保护,中型板块,5054.20,125,4391.780376
880493,0.864846,399106,软件服务,超大板块,16617.04,274,14371.181226
880421,0.862261,399106,广告包装,中小型板块,2810.51,47,2423.392153
880439,0.857017,399106,机械基件,中型板块,3690.03,101,3162.417018


In [41]:
深证影响力 = main_index.groupby(['index'])['related_mv'].sum()
深证影响力

index
399106    288563.588204
Name: related_mv, dtype: float64

### 上证50

In [42]:
# 上证50
index_code = '000016'
one_blk_corr = detail_industry_corr[index_code].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
main_index_raw = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
main_index = main_index_raw.merge(blk_basics,left_on='名称',right_index=True)
main_index.loc[:,'related_mv'] =  main_index.apply(lambda x: x[index_code]*x['blk_mv'],axis=1)
main_index.insert(1,'index',index_code)
main_index

,000016,index,名称,category,blk_mv,count,related_mv
code,,,,,,,
880381,0.808278,000016,白酒,超大板块,42853.54,20,34637.591314
880387,0.782556,000016,家用电器,大板块,13659.28,79,10689.156480
880472,0.763694,000016,证券,超大板块,21999.56,49,16800.922037
880373,0.763024,000016,乳制品,中小型板块,2737.83,16,2089.030673
880473,0.762055,000016,保险,大板块,15020.31,6,11446.302094
880471,0.745858,000016,银行,超大板块,59285.89,42,44218.866998
880375,0.709391,000016,食品,大板块,10395.41,89,7374.409849
880374,0.706972,000016,软饮料,小型板块,649.46,10,459.150102
880447,0.703074,000016,工程机械,中型板块,4301.18,27,3024.047983


In [43]:
上证50影响力 = main_index.groupby(['index'])['related_mv'].sum()
上证50影响力

index
000016    130739.477531
Name: related_mv, dtype: float64

### 中证500

In [44]:
# 中证500
index_code = '000905'
one_blk_corr = detail_industry_corr[index_code].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
main_index_raw = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
main_index = main_index_raw.merge(blk_basics,left_on='名称',right_index=True)
main_index.loc[:,'related_mv'] =  main_index.apply(lambda x: x[index_code]*x['blk_mv'],axis=1)
main_index.insert(1,'index',index_code)
main_index

,000905,index,名称,category,blk_mv,count,related_mv
code,,,,,,,
880432,0.902463,000905,运输设备,中小型板块,2945.98,41,2658.637306
880490,0.885111,000905,通信设备,大板块,11711.94,139,10366.369242
880336,0.879486,000905,化工原料,超大板块,21309.43,227,18741.339252
880456,0.878049,000905,环境保护,中型板块,5054.20,125,4437.833884
880414,0.866893,000905,商贸代理,小型板块,1585.77,27,1374.693644
...,...,...,...,...,...,...,...
880313,0.710894,000905,石油贸易,小型板块,299.22,7,212.713656
880441,0.708823,000905,化工机械,小型板块,909.16,10,644.433119
880391,0.707760,000905,汽车整车,大板块,11043.01,23,7815.803550


In [45]:
中证500影响力 = main_index.groupby(['index'])['related_mv'].sum()
中证500影响力

index
000905    269059.503914
Name: related_mv, dtype: float64

### 创业板指

In [46]:
# 创业板指
index_code = '399006'
one_blk_corr = detail_industry_corr[index_code].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
main_index_raw = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
main_index = main_index_raw.merge(blk_basics,left_on='名称',right_index=True)
main_index.loc[:,'related_mv'] =  main_index.apply(lambda x: x[index_code]*x['blk_mv'],axis=1)
main_index.insert(1,'index',index_code)
main_index

,399006,index,名称,category,blk_mv,count,related_mv
code,,,,,,,
880446,0.911677,399006,电气设备,超大板块,44930.77,263,40962.341694
880492,0.857686,399006,元器件,超大板块,25774.33,252,22106.286661
880445,0.828132,399006,专用机械,大板块,12506.07,218,10356.681527
880448,0.804186,399006,电器仪表,中型板块,7784.65,90,6260.303320
880338,0.804149,399006,塑料,中小型板块,3180.11,68,2557.281524
880336,0.801481,399006,化工原料,超大板块,21309.43,227,17079.093171
880490,0.800163,399006,通信设备,大板块,11711.94,139,9371.465355
880392,0.772676,399006,汽车配件,大板块,13803.69,200,10665.782185
880351,0.764466,399006,矿物制品,中小型板块,1985.87,23,1518.129732


In [47]:
创业影响力 = main_index.groupby(['index'])['related_mv'].sum()
创业影响力

index
399006    191646.276379
Name: related_mv, dtype: float64

### 科创50

In [48]:
# 科创50
index_code = '000688'
one_blk_corr = detail_industry_corr[index_code].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
main_index_raw = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
main_index = main_index_raw.merge(blk_basics,left_on='名称',right_index=True)
main_index.loc[:,'related_mv'] =  main_index.apply(lambda x: x[index_code]*x['blk_mv'],axis=1)
main_index.insert(1,'index',index_code)
main_index

,000688,index,名称,category,blk_mv,count,related_mv
code,,,,,,,
880492,0.898284,000688,元器件,超大板块,25774.33,252,23152.660795
880491,0.879770,000688,半导体,超大板块,16499.84,125,14516.065687
880445,0.854557,000688,专用机械,大板块,12506.07,218,10687.145313
880490,0.841057,000688,通信设备,大板块,11711.94,139,9850.412849
880446,0.823985,000688,电气设备,超大板块,44930.77,263,37022.287927
880338,0.822365,000688,塑料,中小型板块,3180.11,68,2615.210348
880448,0.764575,000688,电器仪表,中型板块,7784.65,90,5951.952617
880336,0.753958,000688,化工原料,超大板块,21309.43,227,16066.419434
880439,0.752551,000688,机械基件,中型板块,3690.03,101,2776.936310


In [49]:
科创50影响力 = main_index.groupby(['index'])['related_mv'].sum()
科创50影响力

index
000688    151120.486092
Name: related_mv, dtype: float64

### 深证成指

In [50]:
index_code = '399001'
one_blk_corr = detail_industry_corr[index_code].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
main_index_raw = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
main_index = main_index_raw.merge(blk_basics,left_on='名称',right_index=True)
main_index.loc[:,'related_mv'] =  main_index.apply(lambda x: x[index_code]*x['blk_mv'],axis=1)
main_index.insert(1,'index',index_code)
main_index

,399001,index,名称,category,blk_mv,count,related_mv
code,,,,,,,
880492,0.898443,399001,元器件,超大板块,25774.33,252,23156.764503
880490,0.876553,399001,通信设备,大板块,11711.94,139,10266.141184
880336,0.863349,399001,化工原料,超大板块,21309.43,227,18397.472094
880448,0.856395,399001,电器仪表,中型板块,7784.65,90,6666.734337
880445,0.844278,399001,专用机械,大板块,12506.07,218,10558.600808
880446,0.842428,399001,电气设备,超大板块,44930.77,263,37850.921280
880493,0.824500,399001,软件服务,超大板块,16617.04,274,13700.747381
880432,0.822299,399001,运输设备,中小型板块,2945.98,41,2422.477861
880421,0.820939,399001,广告包装,中小型板块,2810.51,47,2307.256782


In [51]:
深成指影响力 = main_index.groupby(['index'])['related_mv'].sum()
深成指影响力

index
399001    251565.454544
Name: related_mv, dtype: float64

### 沪深300

In [52]:
index_code = '399300'
one_blk_corr = detail_industry_corr[index_code].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
main_index_raw = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
main_index = main_index_raw.merge(blk_basics,left_on='名称',right_index=True)
main_index.loc[:,'related_mv'] =  main_index.apply(lambda x: x[index_code]*x['blk_mv'],axis=1)
main_index.insert(1,'index',index_code)
main_index

,399300,index,名称,category,blk_mv,count,related_mv
code,,,,,,,
880387,0.829045,399300,家用电器,大板块,13659.28,79,11324.158101
880492,0.786225,399300,元器件,超大板块,25774.33,252,20264.432646
880448,0.785113,399300,电器仪表,中型板块,7784.65,90,6111.826971
880432,0.783936,399300,运输设备,中小型板块,2945.98,41,2309.458847
880490,0.776359,399300,通信设备,大板块,11711.94,139,9092.667541
880381,0.770543,399300,白酒,超大板块,42853.54,20,33020.504725
880421,0.769631,399300,广告包装,中小型板块,2810.51,47,2163.054524
880336,0.765007,399300,化工原料,超大板块,21309.43,227,16301.872094
880472,0.763485,399300,证券,超大板块,21999.56,49,16796.341829


In [53]:
沪深300影响力 = main_index.groupby(['index'])['related_mv'].sum()
沪深300影响力

index
399300    237607.273456
Name: related_mv, dtype: float64

### 主要指数影响力

In [54]:
tt = pd.concat([上证影响力,深证影响力,上证50影响力,中证500影响力,创业影响力,科创50影响力,深成指影响力,沪深300影响力]).sort_values(ascending=False)
pd.concat([index_code_name_map,tt.to_frame()],axis=1).sort_values(by='related_mv',ascending = False)

,名称,related_mv
399106,深证综指,288563.588204
000905,中证500,269059.503914
399001,深证成指,251565.454544
399300,沪深300,237607.273456
000001,上证指数,222242.552518
399006,创业板指,191646.276379
000688,科创50,151120.486092
000016,上证50,130739.477531


## 板块分析

1. 分析主要板块之间相关性
2. 找到趋势启动板块，和之相关的板块
3. 找到趋势板块，和之相关的板块
4. 板块成分股和板块的相关性
5. 板块成分股成正相关，比如 市值排序和相关性成正相关的
6. 板块相关性凌乱的，找出龙头和之相关性高的

### 准备数据

In [55]:
all_stock_basic_data.columns

Index(['代码', '名称', '涨幅%', '涨速%', '现价', '市盈(动)', '行业PE', '细分行业', '所属行业',
       'AB股总市值',
       ...
       'parent_industry', 'child_industry', 'ab_mv', 'circulate_mv', 'blk_mv',
       'count', 'blk_code', 'mv_share', 'circulate_share',
       'circulate_share_rk'],
      dtype='object', length=129)

In [56]:
# all_stock_basic_data

In [57]:
component_share = all_stock_basic_data.filter(['名称','circulate_share','circulate_share_rk'])
component_share

,名称,circulate_share,circulate_share_rk
stock_code,,,
000001,平安银行,4.111164,6.0
001227,兰州银行,0.040549,42.0
002142,宁波银行,3.476780,8.0
002807,江阴银行,0.161371,36.0
002839,张家港行,0.168674,34.0
...,...,...,...
600150,中国船舶,26.634149,2.0
600482,中国动力,14.034183,3.0
600685,中船防务,7.785312,4.0


### 所有板块的总分析

In [215]:
def get_blk_affect_capital(ll,detail_industry_corr,detail_industry_code_name_map,blk_basics):
    result = pd.DataFrame()
    for blk_code in ll:
        one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'})
        one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
        blk_index = one_blk_related_blks.join(blk_basics.reset_index().set_index('blk_code'))
        blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
        blk_index.insert(1,'index',blk_code)
        result = pd.concat([result,blk_index],axis=0)
    return result

In [216]:
big_blk = detail_blk_basics.query('blk_mv>10000').blk_code.to_list()
all_blk_affected_capital = get_blk_affect_capital(big_blk,detail_industry_corr,detail_industry_code_name_map,detail_blk_basics)

In [227]:
all_blk_affected_capital.groupby(['index'])[['related_mv']].sum().sort_values(by='related_mv',ascending=False)\
    .join(detail_blk_basics.reset_index().set_index('blk_code'))

,related_mv,细分行业,blk_mv,count,category
index,,,,,
880445,193315.991046,专用机械,12506.07,218,大板块
880492,193263.831485,元器件,25774.33,252,超大板块
880490,186564.707333,通信设备,11711.94,139,大板块
880336,166194.444887,化工原料,21309.43,227,超大板块
880446,146707.797815,电气设备,44930.77,263,超大板块
880392,141912.013154,汽车配件,13803.69,200,大板块
880493,119905.180978,软件服务,16617.04,274,超大板块
880329,115936.085752,小金属,11713.18,53,大板块
880391,81403.875080,汽车整车,11043.01,23,大板块


### 银行

#### 板块之间关联分析

In [234]:
blk_code = '880471'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='88*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880471,1.000000,880471,银行,59285.89,42,超大板块,59285.890000
880473,0.787302,880471,保险,15020.31,6,大板块,11825.514082


银行和地产有一定的正相关，银行的业务很大部分由地产贡献。

In [131]:
blk影响力1 = blk_index.groupby(['index'])['related_mv'].sum()
blk影响力1

index
880471    71111.404082
Name: related_mv, dtype: float64

In [59]:
# top1_stocks_non_small.loc[['银行']]

In [60]:
# top1_stocks_non_small.loc['银行']
# top1 = top1_stocks_non_small.loc['银行']['代码']
# top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
# pd.concat([detail_industry_data.pivot('close')[[bank_blk_code]].pct_change(),top1_data],axis=1).corr()

#### 全板块关联分析

In [61]:
blk_code = '880471'
component = all_blk_component.loc['银行'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880471,名称,circulate_share,circulate_share_rk
code,,,,
880471,1.000000,NaN,NaN,NaN
600036,0.885209,招商银行,12.161106,3.0
601166,0.871730,兴业银行,6.034016,5.0
600000,0.838790,浦发银行,3.534990,7.0
600919,0.833946,江苏银行,1.815997,13.0
000001,0.827596,平安银行,4.111164,6.0
601658,0.812731,邮储银行,0.859581,21.0
601577,0.760766,长沙银行,0.232248,30.0
601997,0.753046,贵阳银行,0.332305,27.0


### 电气设备

#### 板块之间关联分析

In [237]:
# 电气设备
blk_code = '880446'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index
# blk影响力2 = blk_index.groupby(['index'])['related_mv'].sum()
# blk影响力2

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880446,1.000000,880446,电气设备,44930.77,263.0,超大板块,44930.770000
880445,0.892434,880446,专用机械,12506.07,218.0,大板块,11160.847399
880338,0.879571,880446,塑料,3180.11,68.0,中小型板块,2797.131571
880351,0.850021,880446,矿物制品,1985.87,23.0,中小型板块,1688.031346
880439,0.825018,880446,机械基件,3690.03,101.0,中型板块,3044.342745
000688,0.823985,880446,NaN,NaN,NaN,NaN,NaN
880336,0.822402,880446,化工原料,21309.43,227.0,超大板块,17524.920881
880492,0.815139,880446,元器件,25774.33,252.0,超大板块,21009.661010
880392,0.810632,880446,汽车配件,13803.69,200.0,大板块,11189.713055


In [63]:
blk_mv.loc['电气设备']

blk_mv    44930.77
count       263.00
Name: 电气设备, dtype: float64

In [133]:
blk影响力2 = blk_index.groupby(['index'])['related_mv'].sum()
blk影响力2

index
880446    146707.797815
Name: related_mv, dtype: float64

#### 全板块关联分析

In [64]:
blk_code = '880446'
component = all_blk_component.loc['电气设备'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final[:30]

,880446,名称,circulate_share,circulate_share_rk
code,,,,
880446,1.000000,NaN,NaN,NaN
300750,0.819629,宁德时代,18.026110,1.0
300014,0.806308,亿纬锂能,3.316636,5.0
002074,0.801050,国轩高科,0.858810,21.0
600884,0.796699,杉杉股份,0.883381,20.0
300073,0.764338,当升科技,0.767091,24.0
002249,0.760626,大洋电机,0.208365,76.0
601126,0.756863,四方股份,0.248315,67.0
600580,0.755662,卧龙电驱,0.353566,49.0


### 白酒

#### 板块之间关联分析

In [238]:
# 白酒
blk_code = '880381'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index
# blk影响力3 = blk_index.groupby(['index'])['related_mv'].sum()
# blk影响力3

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880381,1.000000,880381,白酒,42853.54,20,超大板块,42853.540000
880382,0.774980,880381,啤酒,1644.16,7,小型板块,1274.191816
880425,0.703804,880381,旅游服务,4076.12,12,中小型板块,2868.788030


In [136]:
blk影响力3 = blk_index.groupby(['index'])['related_mv'].sum()
blk影响力3

index
880381    46996.519845
Name: related_mv, dtype: float64

#### 全板块关联分析

In [66]:
blk_code = '880381'
component = all_blk_component.loc['白酒'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880381,名称,circulate_share,circulate_share_rk
code,,,,
880381,1.000000,NaN,NaN,NaN
600519,0.942528,贵州茅台,54.494261,1.0
000858,0.908621,五 粮 液,14.589763,2.0
000568,0.902731,泸州老窖,7.450073,4.0
603369,0.822523,今世缘,1.230680,7.0
600809,0.819469,山西汾酒,8.226345,3.0
600702,0.794044,舍得酒业,1.031887,8.0
000799,0.771694,酒鬼酒,0.952943,9.0
002304,0.770399,洋河股份,5.085298,5.0


白酒是典型的正相关板块

### 元器件

In [239]:
# 元器件
blk_code = '880492'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index
# blk影响力4 = blk_index.groupby(['index'])['related_mv'].sum()
# blk影响力4

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880492,1.000000,880492,元器件,25774.33,252.0,超大板块,25774.330000
880490,0.918125,880492,通信设备,11711.94,139.0,大板块,10753.026191
000688,0.898284,880492,NaN,NaN,NaN,NaN,NaN
880445,0.886275,880492,专用机械,12506.07,218.0,大板块,11083.816052
880491,0.861146,880492,半导体,16499.84,125.0,超大板块,14208.771094
880448,0.846808,880492,电器仪表,7784.65,90.0,中型板块,6592.107166
880338,0.824866,880492,塑料,3180.11,68.0,中小型板块,2623.163739
880493,0.817912,880492,软件服务,16617.04,274.0,超大板块,13591.281279
880439,0.815772,880492,机械基件,3690.03,101.0,中型板块,3010.223560


In [138]:
blk影响力4 = blk_index.groupby(['index'])['related_mv'].sum()
blk影响力4

index
880492    193263.831485
Name: related_mv, dtype: float64

#### 全板块关联分析

In [68]:
blk_code = '880492'
component = all_blk_component.loc['元器件'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880492,名称,circulate_share,circulate_share_rk
code,,,,
880492,1.000000,NaN,NaN,NaN
300088,0.852228,长信科技,0.606534,38.0
002273,0.828649,水晶光电,0.615535,36.0
000636,0.820540,风华高科,0.561993,41.0
300460,0.804789,惠伦晶体,0.114106,146.0
...,...,...,...,...
000020,0.245181,深华发Ａ,0.076743,178.0
301282,0.232764,金禄电子,0.045704,207.0
000509,0.169329,华塑控股,0.154883,123.0


In [69]:
blk_mv.loc['元器件']

blk_mv    25774.33
count       252.00
Name: 元器件, dtype: float64

元器件是非正相关板块

### 证券

In [240]:
# 证券
blk_code = '880472'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index
# blk影响力5 = blk_index.groupby(['index'])['related_mv'].sum()
# blk影响力5

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880472,1.000000,880472,证券,21999.56,49,超大板块,21999.560000
880474,0.824514,880472,多元金融,3414.14,30,中小型板块,2815.007503
880473,0.775428,880472,保险,15020.31,6,大板块,11647.167826


In [140]:
blk影响力5 = blk_index.groupby(['index'])['related_mv'].sum()
blk影响力5

index
880472    36461.73533
Name: related_mv, dtype: float64

#### 全板块关联分析

In [71]:
blk_code = '880472'
component = all_blk_component.loc['证券'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880472,名称,circulate_share,circulate_share_rk
code,,,,
880472,1.000000,NaN,NaN,NaN
000166,0.920135,申万宏源,4.107673,8.0
601688,0.917307,华泰证券,4.124764,7.0
600837,0.914106,海通证券,3.650255,9.0
000776,0.911315,广发证券,4.278131,6.0
600369,0.905064,西南证券,0.949428,33.0
601162,0.901387,天风证券,1.130523,29.0
600030,0.889323,中信证券,9.517190,2.0
600061,0.888606,国投资本,1.793263,16.0


### 化工原料

In [241]:
# 化工原料
blk_code = '880336'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index
# blk影响力6 = blk_index.groupby(['index'])['related_mv'].sum()
# blk影响力6

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880336,1.000000,880336,化工原料,21309.43,227.0,超大板块,21309.430000
880329,0.879988,880336,小金属,11713.18,53.0,大板块,10307.457625
880445,0.851470,880336,专用机械,12506.07,218.0,大板块,10648.541116
880338,0.849356,880336,塑料,3180.11,68.0,中小型板块,2701.046874
880351,0.824714,880336,矿物制品,1985.87,23.0,中小型板块,1637.775309
880446,0.822402,880336,电气设备,44930.77,263.0,超大板块,36951.161498
880337,0.820696,880336,农药化肥,7032.39,53.0,中型板块,5771.452019
880320,0.818197,880336,特种钢,1881.14,11.0,中小型板块,1539.144006
880347,0.818007,880336,玻璃,2817.22,20.0,中小型板块,2304.506217


In [143]:
blk影响力6 = blk_index.groupby(['index'])['related_mv'].sum()
blk影响力6

index
880336    166194.444887
Name: related_mv, dtype: float64

#### 全板块关联分析

In [73]:
blk_code = '880336'
component = all_blk_component.loc['化工原料'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)[:20]
blk_component_corr_final

,880336,名称,circulate_share,circulate_share_rk
code,,,,
880336,1.000000,NaN,NaN,NaN
300082,0.803148,奥克股份,0.311130,78.0
601678,0.799626,滨化股份,0.538916,49.0
603026,0.786870,石大胜华,0.990125,20.0
603010,0.762340,万盛股份,0.327695,75.0
300821,0.753740,东岳硅材,0.274151,86.0
600273,0.748738,嘉化能源,0.644738,40.0
002326,0.741407,永太科技,0.795282,30.0
603002,0.731022,宏昌电子,0.158850,120.0


In [74]:
blk_mv.loc['化工原料']

blk_mv    21309.43
count       227.00
Name: 化工原料, dtype: float64

In [142]:
blk影响力6 = blk_index.groupby(['index'])['related_mv'].sum()
blk影响力6

index
880336    166194.444887
Name: related_mv, dtype: float64

### 半导体

In [242]:
# 半导体
blk_code = '880491'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index
# blk影响力7 = blk_index.groupby(['index'])['related_mv'].sum()
# blk影响力7

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880491,1.000000,880491,半导体,16499.84,125.0,超大板块,16499.840000
000688,0.879770,880491,NaN,NaN,NaN,NaN,NaN
880492,0.861146,880491,元器件,25774.33,252.0,超大板块,22195.460991
880490,0.798394,880491,通信设备,11711.94,139.0,大板块,9350.737413
880445,0.739785,880491,专用机械,12506.07,218.0,大板块,9251.804753
880448,0.719090,880491,电器仪表,7784.65,90.0,中型板块,5597.863050
880493,0.716349,880491,软件服务,16617.04,274.0,超大板块,11903.607027
880338,0.714602,880491,塑料,3180.11,68.0,中小型板块,2272.512944


In [145]:
blk影响力7 = blk_index.groupby(['index'])['related_mv'].sum()
blk影响力7

index
880491    77071.826178
Name: related_mv, dtype: float64

#### 全板块关联分析

In [76]:
blk_code = '880491'
component = all_blk_component.loc['半导体'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880491,名称,circulate_share,circulate_share_rk
code,,,,
880491,1.000000,NaN,NaN,NaN
688396,0.861675,华润微,1.326255,20.0
600460,0.856658,士兰微,3.177182,8.0
300623,0.854625,捷捷微电,0.758068,33.0
002371,0.846177,北方华创,9.414637,1.0
...,...,...,...,...
688041,0.268398,海光信息,0.757280,34.0
688401,0.247490,路维光电,0.075152,114.0
300269,0.241938,ST联建,0.126729,98.0


In [77]:
blk_mv.loc['半导体']

blk_mv    16499.84
count       125.00
Name: 半导体, dtype: float64

### 软件服务

In [243]:
# 软件服务
blk_code = '880493'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880493,1.000000,880493,软件服务,16617.04,274.0,超大板块,16617.040000
880489,0.937789,880493,IT设备,3204.47,44.0,中小型板块,3005.117427
880490,0.871147,880493,通信设备,11711.94,139.0,大板块,10202.820456
880494,0.864842,880493,互联网,5077.39,78.0,中型板块,4391.139576
880421,0.827282,880493,广告包装,2810.51,47.0,中小型板块,2325.083813
880492,0.817912,880493,元器件,25774.33,252.0,超大板块,21081.141335
880448,0.786689,880493,电器仪表,7784.65,90.0,中型板块,6124.098751
880452,0.783209,880493,电信运营,3070.59,11.0,中型板块,2404.912435
880432,0.781793,880493,运输设备,2945.98,41.0,中小型板块,2303.145784


#### 全板块关联分析

In [79]:
blk_code = '880493'
component = all_blk_component.loc['软件服务'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880493,名称,circulate_share,circulate_share_rk
code,,,,
880493,1.000000,NaN,NaN,NaN
002368,0.888294,太极股份,0.658120,33.0
002065,0.852320,东华软件,1.001442,19.0
300271,0.849096,华宇软件,0.267135,94.0
300002,0.846761,神州泰岳,0.428055,57.0
...,...,...,...,...
002090,0.253013,金智科技,0.274056,90.0
301195,0.251671,北路智控,0.093037,196.0
002417,0.210803,*ST深南,0.102485,186.0


### 医疗保健

In [244]:
# 医疗保健
blk_code = '880398'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880398,1.000000,880398,医疗保健,16474.48,157,超大板块,16474.480000
880401,0.879243,880398,化学制药,14614.56,136,大板块,12849.752986
880402,0.845672,880398,生物制药,7216.90,73,大板块,6103.131855
880403,0.767659,880398,中成药,8239.00,74,中型板块,6324.746296
880355,0.707825,880398,日用化工,1475.20,16,小型板块,1044.182943


#### 全板块关联分析

In [81]:
blk_code = '880398'
component = all_blk_component.loc['医疗保健'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880398,名称,circulate_share,circulate_share_rk
code,,,,
880398,1.000000,NaN,NaN,NaN
300760,0.744716,迈瑞医疗,21.953409,1.0
600763,0.744044,通策医疗,2.478621,5.0
300463,0.743303,迈克生物,0.499743,45.0
300326,0.724852,凯利泰,0.299676,62.0
...,...,...,...,...
002219,0.156624,恒康医疗,0.785457,27.0
600532,0.017996,*ST未来,0.712314,30.0
301333,0.003960,诺思格,0.069198,133.0


### 化学制药

In [245]:
# 化学制药
blk_code = '880401'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880401,1.000000,880401,化学制药,14614.56,136,大板块,14614.560000
880402,0.898037,880401,生物制药,7216.90,73,大板块,6481.043288
880398,0.879243,880401,医疗保健,16474.48,157,超大板块,14485.075060
880403,0.808384,880401,中成药,8239.00,74,中型板块,6660.276078
880410,0.736071,880401,医药商业,2658.19,33,中小型板块,1956.615898


#### 全板块关联分析

In [83]:
blk_code = '880401'
component = all_blk_component.loc['化学制药'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880401,名称,circulate_share,circulate_share_rk
code,,,,
880401,1.000000,NaN,NaN,NaN
300725,0.780446,药石科技,0.771833,23.0
002019,0.755899,亿帆医药,0.613703,32.0
300558,0.733423,贝达药业,1.204484,18.0
688505,0.731519,复旦张江,0.165451,102.0
...,...,...,...,...
002411,0.185143,*ST必康,1.000235,19.0
600781,0.175955,*ST辅仁,0.069657,127.0
688091,0.166363,上海谊众-U,0.467068,43.0


### 煤炭开采

In [246]:
# 煤炭开采
blk_code = '880302'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880302,1.0,880302,煤炭开采,14166.22,26,大板块,14166.22


#### 全板块关联分析

In [85]:
blk_code = '880302'
component = all_blk_component.loc['煤炭开采'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880302,名称,circulate_share,circulate_share_rk
code,,,,
880302,1.000000,NaN,NaN,NaN
601225,0.908454,陕西煤业,15.480559,2.0
601088,0.905583,中国神华,36.040878,1.0
601898,0.897744,中煤能源,6.537947,4.0
600188,0.877398,兖矿能源,10.534144,3.0
600985,0.871588,淮北矿业,2.583258,8.0
600123,0.866678,兰花科创,1.191920,14.0
601699,0.863090,潞安环能,3.327987,6.0
600971,0.860245,恒源煤电,0.637008,22.0


### 汽车配件

In [247]:
# 汽车配件
blk_code = '880392'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880392,1.000000,880392,汽车配件,13803.69,200.0,大板块,13803.690000
880445,0.872656,880392,专用机械,12506.07,218.0,大板块,10913.492840
880439,0.832954,880392,机械基件,3690.03,101.0,中型板块,3073.626974
880391,0.824483,880392,汽车整车,11043.01,23.0,大板块,9104.770091
880338,0.815158,880392,塑料,3180.11,68.0,中小型板块,2592.292963
880492,0.811884,880392,元器件,25774.33,252.0,超大板块,20925.771001
880446,0.810632,880392,电气设备,44930.77,263.0,超大板块,36422.320674
880351,0.795301,880392,矿物制品,1985.87,23.0,中小型板块,1579.363766
880490,0.779888,880392,通信设备,11711.94,139.0,大板块,9134.003665


#### 全板块关联分析

In [87]:
blk_code = '880392'
component = all_blk_component.loc['汽车配件'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880392,名称,circulate_share,circulate_share_rk
code,,,,
880392,1.000000,NaN,NaN,NaN
000887,0.855672,中鼎股份,1.453307,16.0
002906,0.815265,华阳集团,1.540095,13.0
603197,0.799076,保隆科技,0.693148,32.0
600742,0.790588,一汽富维,0.414889,53.0
...,...,...,...,...
000622,0.193339,恒立实业,0.136775,147.0
000980,0.171796,ST众泰,1.667960,11.0
603201,0.168489,常润股份,0.057521,183.0


### 家用电器

In [256]:
# 家用电器
blk_code = '880387'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880387,1.000000,880387,家用电器,13659.28,79,大板块,13659.280000
880399,0.770826,880387,家居用品,2684.91,59,中小型板块,2069.597857
880340,0.729281,880387,染料涂料,1671.65,28,小型板块,1219.102717
880447,0.725637,880387,工程机械,4301.18,27,中型板块,3121.096668
880374,0.713755,880387,软饮料,649.46,10,小型板块,463.555424
880432,0.712879,880387,运输设备,2945.98,41,中小型板块,2100.128250
880383,0.700803,880387,红黄酒,646.69,10,小型板块,453.202047


#### 全板块关联分析

In [89]:
blk_code = '880387'
component = all_blk_component.loc['家用电器'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final=blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880387,名称,circulate_share,circulate_share_rk
code,,,,
880387,1.000000,NaN,NaN,NaN
000333,0.878319,美的集团,27.639524,1.0
600690,0.818635,海尔智家,12.035847,3.0
002508,0.773744,老板电器,1.728642,9.0
000651,0.733538,格力电器,14.149428,2.0
...,...,...,...,...
300824,0.131244,北鼎股份,0.098029,68.0
605117,0.121131,德业股份,2.307589,8.0
002052,0.117914,*ST同洲,0.131046,63.0


### 专用机械

In [248]:
# 专用机械
blk_code = '880445'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880445,1.000000,880445,专用机械,12506.07,218.0,大板块,12506.070000
880439,0.935674,880445,机械基件,3690.03,101.0,中型板块,3452.663999
880446,0.892434,880445,电气设备,44930.77,263.0,超大板块,40097.765925
880338,0.888568,880445,塑料,3180.11,68.0,中小型板块,2825.742701
880492,0.886275,880445,元器件,25774.33,252.0,超大板块,22843.141976
880351,0.873902,880445,矿物制品,1985.87,23.0,中小型板块,1735.455277
880392,0.872656,880445,汽车配件,13803.69,200.0,大板块,12045.868285
000688,0.854557,880445,NaN,NaN,NaN,NaN,NaN
880336,0.851470,880445,化工原料,21309.43,227.0,超大板块,18144.336431


#### 全板块关联分析

In [91]:
blk_code = '880445'
component = all_blk_component.loc['专用机械'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880445,名称,circulate_share,circulate_share_rk
code,,,,
880445,1.000000,NaN,NaN,NaN
300457,0.861421,赢合科技,1.054368,19.0
300227,0.782313,光韵达,0.222292,112.0
300263,0.774679,隆华科技,0.409641,62.0
688151,0.760093,华强科技,0.116503,166.0
...,...,...,...,...
000856,0.263352,冀东装备,0.162641,134.0
688400,0.186715,凌云光,0.171357,130.0
001336,0.154934,楚环科技,0.046857,206.0


### 建筑工程

In [249]:
# 建筑工程
blk_code = '880477'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880477,1.000000,880477,建筑工程,12714.53,115,大板块,12714.530000
880346,0.834144,880477,水泥,3055.09,23,中小型板块,2548.383874
880484,0.749371,880477,区域地产,4447.82,53,中型板块,3333.066800
880319,0.744055,880477,普钢,4712.81,25,中型板块,3506.591231
880485,0.738725,880477,园区开发,1603.99,15,小型板块,1184.907463
880321,0.716579,880477,钢加工,1292.47,23,小型板块,926.156249
880466,0.706187,880477,路桥,1803.13,20,中小型板块,1273.347577


#### 全板块关联分析

In [93]:
blk_code = '880477'
component = all_blk_component.loc['建筑工程'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880477,名称,circulate_share,circulate_share_rk
code,,,,
880477,1.000000,NaN,NaN,NaN
601186,0.908079,中国铁建,6.550222,4.0
601800,0.875184,中国交建,7.169671,3.0
600170,0.873776,上海建工,1.855908,11.0
601390,0.859060,中国中铁,8.840830,2.0
...,...,...,...,...
300844,0.205008,山水比德,0.032325,111.0
300536,0.199772,农尚环境,0.393723,44.0
600769,0.187489,祥龙电业,0.224153,64.0


### 通信设备

In [250]:
# 通信设备
blk_code = '880490'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880490,1.000000,880490,通信设备,11711.94,139.0,大板块,11711.940000
880492,0.918125,880490,元器件,25774.33,252.0,超大板块,23664.059544
880493,0.871147,880490,软件服务,16617.04,274.0,超大板块,14475.883212
880489,0.866117,880490,IT设备,3204.47,44.0,中小型板块,2775.444343
880445,0.850702,880490,专用机械,12506.07,218.0,大板块,10638.938550
880448,0.841179,880490,电器仪表,7784.65,90.0,中型板块,6548.284282
000688,0.841057,880490,NaN,NaN,NaN,NaN,NaN
880432,0.832310,880490,运输设备,2945.98,41.0,中小型板块,2451.968554
880439,0.826555,880490,机械基件,3690.03,101.0,中型板块,3050.013042


#### 全板块关联分析

In [95]:
blk_code = '880490'
component = all_blk_component.loc['通信设备'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880490,名称,circulate_share,circulate_share_rk
code,,,,
880490,1.000000,NaN,NaN,NaN
300136,0.859210,信维通信,1.082912,20.0
600498,0.842089,烽火通信,1.342903,16.0
300183,0.834390,东软载波,0.386699,58.0
002281,0.811983,光迅科技,0.937846,23.0
...,...,...,...,...
300913,0.345089,兆龙互连,0.066855,137.0
000889,0.317052,ST中嘉,0.178963,103.0
600260,0.255095,*ST凯乐,0.228485,86.0


### 小金属	

In [251]:
# 小金属
blk_code = '880329'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880329,1.000000,880329,小金属,11713.18,53,大板块,11713.180000
880336,0.879988,880329,化工原料,21309.43,227,超大板块,18752.042292
880351,0.798035,880329,矿物制品,1985.87,23,中小型板块,1584.793391
880320,0.786947,880329,特种钢,1881.14,11,中小型板块,1480.357072
880445,0.774667,880329,专用机械,12506.07,218,大板块,9688.044813
880337,0.772378,880329,农药化肥,7032.39,53,中型板块,5431.666062
880327,0.765302,880329,铅锌,1442.54,14,小型板块,1103.979280
880446,0.759612,880329,电气设备,44930.77,263,超大板块,34129.962018
880338,0.739957,880329,塑料,3180.11,68,中小型板块,2353.143898


#### 全板块关联分析

In [97]:
blk_code = '880329'
component = all_blk_component.loc['小金属'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880329,名称,circulate_share,circulate_share_rk
code,,,,
880329,1.000000,NaN,NaN,NaN
300618,0.827998,寒锐钴业,1.094323,21.0
002240,0.821022,盛新锂能,3.062704,10.0
002466,0.816098,天齐锂业,13.095163,1.0
000831,0.804949,五矿稀土,2.064256,16.0
600111,0.795683,北方稀土,8.932843,3.0
600392,0.791473,盛和资源,2.344965,13.0
000962,0.778528,东方钽业,0.366596,36.0
000762,0.776889,西藏矿业,2.195134,15.0


### 石油开采

In [252]:
# 石油开采
blk_code = '880311'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880311,1.000000,880311,石油开采,11587.12,20,中型板块,11587.120000
880312,0.826338,880311,石油加工,6125.42,11,中型板块,5061.667938


#### 全板块关联分析

In [99]:
blk_code = '880311'
component = all_blk_component.loc['石油开采'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880311,名称,circulate_share,circulate_share_rk
code,,,,
880311,1.000000,NaN,NaN,NaN
601857,0.984796,中国石油,75.042116,1.0
601808,0.850667,中海油服,3.661307,3.0
600938,0.825496,中国海油,2.076875,6.0
600583,0.804174,海油工程,1.720876,7.0
600871,0.802452,石化油服,2.201582,5.0
300191,0.763492,潜能恒信,0.381199,12.0
603619,0.761680,中曼石油,0.771892,10.0
600968,0.719201,海油发展,2.447632,4.0


### 汽车整车

In [253]:
# 汽车整车
blk_code = '880391'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880391,1.000000,880391,汽车整车,11043.01,23,大板块,11043.010000
880392,0.824483,880391,汽车配件,13803.69,200,大板块,11380.902839
880393,0.719794,880391,汽车服务,849.52,14,小型板块,611.479314
880446,0.702201,880391,电气设备,44930.77,263,超大板块,31550.413788
880445,0.701019,880391,专用机械,12506.07,218,大板块,8766.988721
880492,0.700351,880391,元器件,25774.33,252,超大板块,18051.080418


#### 全板块关联分析

In [101]:
blk_code = '880391'
component = all_blk_component.loc['汽车整车'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880391,名称,circulate_share,circulate_share_rk
code,,,,
880391,1.000000,NaN,NaN,NaN
601633,0.876177,长城汽车,16.034578,2.0
002594,0.831608,比亚迪,28.054308,1.0
601238,0.829698,广汽集团,8.510904,5.0
000625,0.770929,长安汽车,9.041557,4.0
600733,0.733564,北汽蓝谷,2.398440,8.0
600104,0.692531,上汽集团,15.891138,3.0
601127,0.691725,赛力斯,5.898573,6.0
600418,0.681363,江淮汽车,2.866610,7.0


### 食品

In [254]:
# 食品
blk_code = '880375'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880375,1.000000,880375,食品,10395.41,89,大板块,10395.410000
880373,0.794374,880375,乳制品,2737.83,16,中小型板块,2174.862248
880374,0.793427,880375,软饮料,649.46,10,小型板块,515.298980
880383,0.768652,880375,红黄酒,646.69,10,小型板块,497.079804
880355,0.703528,880375,日用化工,1475.20,16,小型板块,1037.844811


#### 全板块关联分析

In [103]:
blk_code = '880446'
component = all_blk_component.loc['食品'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880446,名称,circulate_share,circulate_share_rk
code,,,,
880446,1.000000,NaN,NaN,NaN
300829,0.546466,金丹科技,0.250014,59.0
688089,0.464269,嘉必优,0.257518,57.0
603079,0.461865,圣达生物,0.213941,63.0
603020,0.447199,爱普股份,0.406622,45.0
...,...,...,...,...
603170,0.030415,宝立食品,0.089655,81.0
603696,0.028574,安记食品,0.199319,69.0
002216,-0.034663,三全食品,0.945610,20.0


### 水力发电

In [255]:
# 水力发电
blk_code = '880306'
one_blk_corr = detail_industry_corr[[blk_code]].rename(columns = {blk_code:'corr'}).filter(regex='880*',axis=0)
one_blk_related_blks = one_blk_corr.query('corr > 0.7').sort_values(by = 'corr',ascending= False)
blk_index = one_blk_related_blks.join(detail_blk_basics.reset_index().set_index('blk_code'))
blk_index.loc[:,'related_mv'] =  blk_index.apply(lambda x: x['corr']*x['blk_mv'],axis=1)
blk_index.insert(1,'index',blk_code)
blk_index

,corr,index,细分行业,blk_mv,count,category,related_mv
code,,,,,,,
880306,1.0,880306,水力发电,9230.41,18,中型板块,9230.41


#### 全板块关联分析

In [105]:
blk_code = '880306'
component = all_blk_component.loc['水力发电'].values[0]
blk_component_data = QA.QA_fetch_stock_day_adv(component,'2022-01-01','2022-09-08')
blk_component_corr = pd.concat([detail_industry_data.pivot('close')[[blk_code]].pct_change(),blk_component_data.pivot('close').pct_change()],axis=1).corr()
blk_component_corr_final = blk_component_corr[[blk_code]].join(component_share).sort_values(by=blk_component_corr.index.values[0],ascending=0)
blk_component_corr_final

,880306,名称,circulate_share,circulate_share_rk
code,,,,
880306,1.000000,NaN,NaN,NaN
600900,0.897582,长江电力,57.800683,1.0
600025,0.778317,华能水电,14.099049,2.0
600674,0.765216,川投能源,5.959215,4.0
600886,0.689120,国投电力,8.324007,3.0
600236,0.659329,桂冠电力,5.200636,5.0
600310,0.578860,桂东电力,0.425333,14.0
600505,0.559406,西昌电力,0.321546,17.0
000791,0.542676,甘肃电投,0.764430,9.0


In [106]:
# 龙头
blk_component_corr_final.query('circulate_share_rk <2')

,880306,名称,circulate_share,circulate_share_rk
code,,,,
600900,0.897582,长江电力,57.800683,1.0


### top1龙头汇总

In [ ]:
top1_stocks_non_small

## 申万子行业

In [22]:
child_industry = pd.read_csv('申万子行业.csv')
child_industry.sort_values('代码')

,代码,名称
54,880301,煤炭
15,880305,电力
17,880310,石油
44,880318,钢铁
43,880324,有色
22,880330,化纤
27,880335,化工
48,880344,建材
31,880350,造纸
55,880351,矿物制品


In [13]:
child_industry_code = child_industry['代码'].astype(str).to_list()
child_industry_code[:5]

['880431', '880490', '880494', '880492', '880418']

In [14]:
child_industry_data = QA.QA_fetch_index_day_adv(child_industry_code,'2022-01-01','2022-09-02')

In [37]:
child_industry_data.data.head()

open    close     high      low       vol        amount  \
date       code                                                                 
2022-01-04 880301   931.47   920.47   943.68   914.04  210057.0  1.968611e+10   
           880305  1938.98  1901.07  1947.99  1897.17  525302.0  3.565899e+10   
           880310   693.05   696.31   697.96   690.91  156730.0  7.833487e+09   
           880318  1105.75  1114.31  1118.64  1105.75  263546.0  1.546208e+10   
           880324  1048.57  1026.31  1049.79  1018.67  375823.0  6.011190e+10   

                   up_count  down_count    date_stamp    volume  
date       code                                                  
2022-01-04 880301        30           4  1.641226e+09  210057.0  
           880305        28          42  1.641226e+09  525302.0  
           880310        28           7  1.641226e+09  156730.0  
           880318        46           8  1.641226e+09  263546.0  
           880324        72          43  1.641226e+09  375823.0

In [51]:
# 
child_industry_pivot = child_industry_data.pivot('close')
child_industry_pivot.head()

code,880301,880305,880310,880318,880324,880330,880335,880344,880350,880351,...,880474,880476,880482,880489,880490,880491,880492,880493,880494,880497
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,920.47,1901.07,696.31,1114.31,1026.31,2784.80,2284.52,1789.53,1522.10,2289.38,...,1190.73,1337.87,1350.15,1664.03,1703.57,3399.73,3041.77,2789.40,2457.76,1114.81
2022-01-05,907.28,1846.79,699.07,1095.91,1001.69,2790.02,2234.56,1768.21,1499.92,2212.49,...,1183.91,1320.62,1354.78,1655.08,1674.04,3257.17,2933.95,2761.13,2483.45,1099.13
2022-01-06,907.51,1829.69,700.46,1123.51,1012.70,2858.15,2252.79,1800.38,1521.16,2210.11,...,1185.69,1365.43,1349.27,1653.45,1671.06,3241.99,2915.05,2725.85,2451.54,1111.73
2022-01-07,910.09,1803.95,726.81,1140.37,992.23,2828.08,2209.04,1822.83,1505.44,2155.91,...,1194.68,1380.33,1383.00,1618.84,1650.08,3192.75,2884.19,2672.64,2391.68,1102.10
2022-01-10,927.64,1795.26,727.75,1136.22,1000.28,2848.66,2221.10,1823.01,1514.43,2138.94,...,1198.05,1393.29,1400.01,1667.57,1663.25,3171.70,2899.51,2711.83,2396.91,1108.53


In [61]:
child_industry_corr = child_industry_pivot.pct_change().corr()

In [121]:
child_industry_corr.head()

code,880301,880305,880310,880318,880324,880330,880335,880344,880350,880351,...,880474,880476,880482,880489,880490,880491,880492,880493,880494,880497
code,,,,,,,,,,,,,,,,,,,,,
880301,1.000000,0.457005,0.672686,0.508798,0.479966,0.331723,0.384374,0.371876,0.281436,0.177597,...,0.379791,0.442363,0.418174,0.244306,0.169642,0.016649,0.148080,0.203849,0.231597,0.318186
880305,0.457005,1.000000,0.466193,0.705074,0.555050,0.473926,0.579693,0.677795,0.601600,0.447779,...,0.678953,0.659184,0.532161,0.500336,0.521441,0.338104,0.477805,0.506320,0.530488,0.597552
880310,0.672686,0.466193,1.000000,0.584882,0.502954,0.365056,0.412669,0.428355,0.397470,0.183294,...,0.515422,0.539637,0.465776,0.233717,0.225614,0.060220,0.196128,0.247566,0.299057,0.324592
880318,0.508798,0.705074,0.584882,1.000000,0.727472,0.620702,0.707699,0.823887,0.720836,0.553422,...,0.744609,0.763308,0.651465,0.606746,0.611808,0.404659,0.548855,0.622044,0.654952,0.683499
880324,0.479966,0.555050,0.502954,0.727472,1.000000,0.635683,0.889058,0.620745,0.671856,0.755497,...,0.583304,0.450880,0.359680,0.636690,0.683371,0.570895,0.702771,0.657808,0.598720,0.627190


In [103]:
child_industry.loc[:,('blk_code')] = child_industry['代码'].astype(str)
child_industry_code_name = child_industry.drop('代码',axis=1).set_index('blk_code')

In [105]:
# 电气设备
one_blk_corr = child_industry_corr[supper_big_industry_code[0]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880446,名称
code,,
880446,1.000000,电气设备
880440,0.876329,工业机械
880351,0.843018,矿物制品
880492,0.814913,元器件
880437,0.809383,通用机械
880335,0.801402,化工
880390,0.787617,汽车类
880490,0.748017,通信设备
880448,0.735565,电器仪表


In [120]:
# 酿酒
one_blk_corr = child_industry_corr[supper_big_industry_code[1]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.6].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880380,名称
code,,
880380,1.000000,酿酒
880372,0.699610,食品饮料
880387,0.650577,家用电器
880355,0.618630,日用化工


In [112]:
# 化工
one_blk_corr = child_industry_corr[supper_big_industry_code[2]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880335,名称
code,,
880335,1.000000,化工
880324,0.889058,有色
880440,0.864302,工业机械
880351,0.817956,矿物制品
880456,0.805736,环境保护
880437,0.803928,通用机械
880446,0.801402,电气设备
880492,0.789784,元器件
880414,0.783309,商贸代理


In [115]:
# 医药
one_blk_corr = child_industry_corr[supper_big_industry_code[3]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.65].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880400,名称
code,,
880400,1.000000,医药
880398,0.888376,医疗保健
880406,0.765079,商业连锁
880372,0.691861,食品饮料
880350,0.680243,造纸
880490,0.673468,通信设备
880335,0.672085,化工
880355,0.668702,日用化工
880494,0.667502,互联网


In [116]:
# 汽车类
one_blk_corr = child_industry_corr[supper_big_industry_code[4]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.65].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880390,名称
code,,
880390,1.000000,汽车类
880440,0.837093,工业机械
880492,0.792099,元器件
880446,0.787617,电气设备
880437,0.785773,通用机械
880335,0.764853,化工
880351,0.762659,矿物制品
880490,0.753577,通信设备
880432,0.706670,运输设备


In [119]:
# 元器件
one_blk_corr = child_industry_corr[supper_big_industry_code[5]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880492,名称
code,,
880492,1.000000,元器件
880490,0.917792,通信设备
880440,0.889661,工业机械
880491,0.861494,半导体
880448,0.847548,电器仪表
880493,0.820136,软件服务
880437,0.816570,通用机械
880446,0.814913,电气设备
880489,0.796969,IT设备


In [122]:
# 半导体
one_blk_corr = child_industry_corr['880491']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880491,名称
code,,
880491,1.000000,半导体
880492,0.861494,元器件
880490,0.800875,通信设备


In [123]:
# 热门板块关联分析
one_blk_corr = child_industry_corr['880302']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

KeyError: '880302'